# `pandapower` Power Flow (PF) Examples
This is an introduction to the usage of the `pandapower` **power flow** for a 3-bus system.
This example is adapted from the [`pandapower` tutorial for PF](https://github.com/e2nIEE/pandapower/blob/develop/tutorials/minimal_example.ipynb). The installation of `pandapower` is included at the bottom of this script. 

## Creating a Power System

We consider the following simple 3-bus example network as a minimal example:


![](3bus-system.png)


The above network can be created in pandapower as follows:

Import and create a network in `pandapower`

In [1]:
import pandapower as pp

In [2]:
net = pp.create_empty_network() # create empty net

First, create 3 buses. In `pandapower` these are just empty nodes that we have to fill with something, like a generator, a load, or an external grid. 

In [3]:
bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1") # Low voltage grid/distribution grid at 20 kV
bus2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2") # Low voltage grid/distribution grid at 400 V
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3") # Low voltage grid/distribution grid at 400 V

In [4]:
net.bus

,name,vn_kv,type,zone,in_service
0,Bus 1,20.0,b,None,True
1,Bus 2,0.4,b,None,True
2,Bus 3,0.4,b,None,True


Create a load of **100 kW** and **50kVar**.

In [5]:
load = pp.create_load(net, element=0, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")

Now we connect the bus 2 and bus 3 with an AC line of a standard type. Other types are found [here](https://pandapower.readthedocs.io/en/v2.13.1/std_types/basic.html#lines). 

In [6]:
line = pp.create_line_from_parameters(net, from_bus=bus2, to_bus=bus3, length_km=0.1, r_ohm_per_km=0.624, x_ohm_per_km=0.083, c_nf_per_km=210, max_i_ka=0.142, name='Line')
#line = pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=0.1, std_type='NAYY 4x50 SE', name="Line")

In [7]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line,None,1,2,0.1,0.624,0.083,210.0,0.0,0.142,1.0,1,None,True


Add the *external grid* (`ext_grid`) at bus 1

In [8]:
eg = pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, name="Grid Connection")

In [9]:
net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service
0,Grid Connection,0,1.02,0.0,1.0,True


And the transformer between bus 1 and bus 2. A list of standard types of transformers is given [here](https://pandapower.readthedocs.io/en/v2.13.1/std_types/basic.html#transformers).

In [10]:
trafo = pp.create_transformer(net, hv_bus=bus1, lv_bus=bus2, std_type="0.4 MVA 20/0.4 kV", name="Trafo")

In [11]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,shift_degree,tap_side,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,Trafo,0.4 MVA 20/0.4 kV,0,1,0.4,20.0,0.4,6.0,1.425,1.35,0.3375,150.0,hv,0,-2,2,2.5,0.0,0,False,1,1.0,True


## Power Flow
We now run a power flow:

In [12]:
pp.runpp(net)

And check out at the results for buses, lines an transformers:

In [13]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.020000,0.000000,-0.1071,-0.052668
1,1.008849,-0.758789,0.0000,0.000000
2,0.965668,0.082982,0.1000,0.050000


In [14]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.105228,0.050694,-0.1,-0.05,0.005228,0.000694,0.167111,0.167112,0.167112,1.008849,-0.758789,0.965668,0.082982,117.684235


In [15]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.1071,0.052668,-0.105228,-0.050694,0.001872,0.001974,0.003378,0.167111,1.02,0.0,1.008849,-0.758789,29.252296


## A simple example with a generator that can be controlled

In [16]:
net = pp.create_empty_network() # create empty net

bus2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2") # Low voltage grid/distribution grid at 400 V
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3") # Low voltage grid/distribution grid at 400 V

#                                                         0.100           0.           0.120
gen = pp.create_gen(net, bus=bus2, name="Generator", p_mw=0.100, min_p_mw=0., max_p_mw=0.120, controllable=True, slack=True)

#                                         0.110
load = pp.create_load(net, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")
    
#                                                                0.5   is the limit!
line = pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=0.1, std_type='NAYY 4x50 SE', name="Line")

In [17]:
pp.runpp(net)  # switch to pp.runopp(net) to not surpass the maximum power.

Let us check the generator

In [18]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.105498,0.05071,0.0,1.0


And check the load (which is static)

In [19]:
net.res_load

,p_mw,q_mvar
0,0.1,0.05


And let us check the line loading

In [20]:
net.res_line
#net.line["max_loading_percent"] = 100  # set a limit for the line loading

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.105498,0.05071,-0.1,-0.05,0.005498,0.00071,0.16895,0.168951,0.168951,1.0,0.0,0.955154,0.892315,118.9796


### Installation and others

To use `pandapower` it is advised to have `anaconda`/`miniconda` installed. Then either:
```
pip install pandapower
```
or
```
conda install pandapower
```
is enough.

This script was adapted from [`pandapower` tutorial for PF](https://github.com/e2nIEE/pandapower/blob/develop/tutorials/minimal_example.ipynb) for the *FYS377 Digital Power Systems*, by *Heidi S. Nygård*, NMBU. Adapted by Leonardo Rydin Gorjão. 2023.